# Pandas - Combinando datos
Inteligencia Artificial - Facundo A. Lucianna - CEIA - FIUBA

En el proceso de armar nuestros dataset para entrenar modelos de Aprendizaje Automático, muchas veces debemos combinar datos de diferentes fuentes. Pandas nos da varias herramientas que nos permiten hacer combinación de ellos.

In [1]:
import pandas as pd

In [2]:
# Creemos el siguiente DataFrame con datos sintéticos

def create_df(columns, index):
    
    lista_generica = []
    for i in index:
        row_list = [f"{col}{i}" for col in columns]
        lista_generica.append(row_list)
    return pd.DataFrame(lista_generica, columns = columns, index=index)

df = create_df(['A', 'B', 'C'], range(4))

In [3]:
df

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3


## Concatenando

Pandas tiene una función de concatenación muy parecida a la de NumPy `np.concatenate`, pero tiene mas opciones específicas para DataFrames y series, la función es `pd.concat()`.

Empecemos con series:

In [4]:
# Concatenamos una serie encima de la otra
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [5]:
# Concatenamos una serie encima de la otra, obsérvese los índices
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[1, 1, 1])
ser_concat = pd.concat([ser1, ser2])
ser_concat

1    A
2    B
3    C
1    D
1    E
1    F
dtype: object

In [6]:
ser_concat.loc[1]

1    A
1    D
1    E
1    F
dtype: object

En un DataFrame tenemos mas variedad de concatenación:

In [7]:
df1 = create_df(['A', 'B'], [1, 2])
df2 = create_df(['A', 'B'], [3, 4])

In [8]:
df1

,A,B
1,A1,B1
2,A2,B2


In [9]:
df2

,A,B
3,A3,B3
4,A4,B4


In [10]:
pd.concat([df1, df2])

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


Por defecto, la concatenación se realiza por filas dentro del DataFrame (es decir, axis=0). `pd.concat` permite especificar un eje a lo largo del cual se llevará a cabo la concatenación:

In [11]:
df3 = create_df(['A', 'B'], [0, 1])
df4 = create_df(['C', 'D'], [0, 1])

In [12]:
df3

,A,B
0,A0,B0
1,A1,B1


In [13]:
df4

,C,D
0,C0,D0
1,C1,D1


Podemos usar tanto `axis=1` como `axis='columns'` que es más fácil de entender:

In [14]:
pd.concat([df3, df4], axis='columns')

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1


La concatenación en Pandas conserva los índices, incluso si el resultado tiene índices duplicados:

In [15]:
x = create_df(['A', 'B'], [0, 1])
y = create_df(['A', 'B'], [0, 1])

In [16]:
x

,A,B
0,A0,B0
1,A1,B1


In [17]:
y

,A,B
0,A0,B0
1,A1,B1


In [18]:
pd.concat([x, y])

,A,B
0,A0,B0
1,A1,B1
0,A0,B0
1,A1,B1


Nótese la repetición de los índices, algo que es válido en un DataFrame pero en general y principalmente en DataFrame no es deseable. `pd.concat()` nos da algunas herramientas para evitar esto.

**Generando un error**: Una forma de verificar que los índices en el resultado de `pd.concat()` no se superponen es especificar el argumento `verify_integrity`. Con ponerlo en `True`, la concatenación generará una excepción si hay índices duplicados:

In [19]:
pd.concat([x, y], verify_integrity=True)

ValueError: Indexes have overlapping values: Index([0, 1], dtype='int64')

In [20]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Index([0, 1], dtype='int64')


**Ignorando los índices**: A veces, el índice en sí mismo no importa y se prefiere que se ignore. Esta opción se puede especificar utilizando el argumento `ignore_index`:

In [21]:
pd.concat([x, y], ignore_index=True)

,A,B
0,A0,B0
1,A1,B1
2,A0,B0
3,A1,B1


**Generando un Multiíndice**: Otra opción es utilizar la opción `keys` para especificar una etiqueta para las fuentes de datos:

In [22]:
pd.concat([x, y], keys=['x', 'y'])

A   B
x 0  A0  B0
  1  A1  B1
y 0  A0  B0
  1  A1  B1

En los ejemplos que vimos, concatenamos DataFrames con nombres de columnas compartidos. En la práctica, los datos de diferentes fuentes pueden tener conjuntos diferentes de nombres de columnas, y `pd.concat` ofrece varias opciones en este caso:

In [23]:
df1 = create_df(['A','B','C'], [1, 2])
df2 = create_df(['B','C','D'], [3, 4])

In [24]:
pd.concat([df1, df2])

,A,B,C,D
1,A1,B1,C1,NaN
2,A2,B2,C2,NaN
3,NaN,B3,C3,D3
4,NaN,B4,C4,D4


Por defecto, las entradas para las cuales no hay datos disponibles se llenan con valores `NaN`. Podemos cambiar este comportamiento, especificando una de varias opciones para el parámetro `join` de la función `pd.concatenate`. 

Por defecto, se realiza una unión (`join='outer'`), pero podemos cambiar esto a una intersección de las columnas usando `join='inner'`:

In [25]:
pd.concat([df1, df2], join='inner')

,B,C
1,B1,C1
2,B2,C2
3,B3,C3
4,B4,C4


In [26]:
df1 = create_df(['A', 'B'], [1, 2])
df2 = create_df(['A', 'B'], [3, 4])

## Merge y join

Una característica importantísima de Pandas son sus operaciones de join y merge de alto rendimiento en memoria. La forma principal para esto es la función `pd.merge()`.

El comportamiento implementado en `pd.merge()` es un subconjunto de lo que se conoce como álgebra relacional, que es un conjunto formal de reglas para manipular datos relacionales y constituye la base conceptual de las operaciones disponibles en la mayoría de las bases de datos. 

Pandas implementa varios de estos bloques de construcción fundamentales en la función `pd.merge()`.

### Categorías de joins

La función `pd.merge()` implementa varios tipos de joins: el **join uno a uno**, **muchos a uno** y **muchos a muchos**. 

**join uno a uno**: El tipo más simple de merge, que en muchos aspectos es muy similar a la concatenación por columnas.

In [27]:
columns1 = ['Nombre completo', 'Fecha Nacimiento'] 
columns2 = ['Nombre completo', 'Ciudad', 'Profesion']

per_1a = ["Clotilde Acosta", "3 octubre de 1940"]
per_2a = ["Rosario Bléfari", "24/12/65"]
per_3a = ["Norberto Carredegoas", "1936-04-12"]
per_4a = ["Bárbara Torres", "11-04-1973"]
per_5a = ["Eugenio Weinbaum", "17/08/1961"]

per_1b = ["Norberto Carredegoas", "San Antonio Oeste"]
per_2b = ["Rosario Bléfari",  "MDP", "Cantante"]
per_3b = ["Clotilde Acosta", "Mar del Plata", "Actriz"]
per_4b = ["Bárbara Torres", "M del plata", "Actuación"]
per_5b = ["Eugenio Weinbaum", "MDQ", "Conductor"]

personajes1 = [per_1a, per_2a, per_3a, per_4a, per_5a]
personajes2 = [per_1b, per_2b, per_3b, per_4b, per_5b]

df1 = pd.DataFrame(personajes1, columns = columns1)
df2 = pd.DataFrame(personajes2, columns = columns2)

In [28]:
df1

,Nombre completo,Fecha Nacimiento
0,Clotilde Acosta,3 octubre de 1940
1,Rosario Bléfari,24/12/65
2,Norberto Carredegoas,1936-04-12
3,Bárbara Torres,11-04-1973
4,Eugenio Weinbaum,17/08/1961


In [29]:
df2

,Nombre completo,Ciudad,Profesion
0,Norberto Carredegoas,San Antonio Oeste,None
1,Rosario Bléfari,MDP,Cantante
2,Clotilde Acosta,Mar del Plata,Actriz
3,Bárbara Torres,M del plata,Actuación
4,Eugenio Weinbaum,MDQ,Conductor


In [30]:
df3 = pd.merge(df1, df2)
df3

,Nombre completo,Fecha Nacimiento,Ciudad,Profesion
0,Clotilde Acosta,3 octubre de 1940,Mar del Plata,Actriz
1,Rosario Bléfari,24/12/65,MDP,Cantante
2,Norberto Carredegoas,1936-04-12,San Antonio Oeste,None
3,Bárbara Torres,11-04-1973,M del plata,Actuación
4,Eugenio Weinbaum,17/08/1961,MDQ,Conductor


La función `pd.merge()` reconoce que cada `DataFrame` tiene la columna *Nombre completo* y realiza automáticamente el join utilizando esta columna como `llaves`. El resultado del `merge` es un nuevo DataFrame que combina la información de las dos entradas. 

Observa que el orden de las entradas en cada columna no necesariamente se mantiene, en este caso, el orden de la columna *Nombre completo* difiere entre df1 y df2, y la función `pd.merge()` tiene en cuenta esto correctamente. Además, en general el `merge` descarta el índice.

**Joins de muchos a uno**: Los joins de muchos a uno son aquellos en los que una de las columnas clave contiene entradas duplicadas. El DataFrame resultante conservará esas entradas duplicadas según corresponda. 

In [31]:
columns1 = ['Nombre completo', 'Fecha Nacimiento'] 
columns2 = ['Nombre completo', 'Profesion']

per_1a = ["Clotilde Acosta", "3 octubre de 1940"]
per_2a = ["Rosario Bléfari", "24/12/65"]
per_3a = ["Norberto Carredegoas", "1936-04-12"]
per_4a = ["Bárbara Torres", "11-04-1973"]
per_5a = ["Eugenio Weinbaum", "17/08/1961"]

per_1b = ["Norberto Carredegoas", "Actor"]
per_2b = ["Norberto Carredegoas", "Conductor"]
per_3b = ["Norberto Carredegoas", "Publicista"]
per_4b = ["Rosario Bléfari",  "Cantante"]
per_5b = ["Clotilde Acosta", "Actriz"]
per_6b = ["Bárbara Torres", "Actuación"]
per_7b = ["Eugenio Weinbaum", "Conductor"]
per_8b = ["Eugenio Weinbaum", "Actor"]

personajes1 = [per_1a, per_2a, per_3a, per_4a, per_5a]
personajes2 = [per_1b, per_2b, per_3b, per_4b, per_5b, per_6b, per_7b, per_8b]

df1 = pd.DataFrame(personajes1, columns = columns1)
df2 = pd.DataFrame(personajes2, columns = columns2)

In [32]:
df1

,Nombre completo,Fecha Nacimiento
0,Clotilde Acosta,3 octubre de 1940
1,Rosario Bléfari,24/12/65
2,Norberto Carredegoas,1936-04-12
3,Bárbara Torres,11-04-1973
4,Eugenio Weinbaum,17/08/1961


In [33]:
df2

,Nombre completo,Profesion
0,Norberto Carredegoas,Actor
1,Norberto Carredegoas,Conductor
2,Norberto Carredegoas,Publicista
3,Rosario Bléfari,Cantante
4,Clotilde Acosta,Actriz
5,Bárbara Torres,Actuación
6,Eugenio Weinbaum,Conductor
7,Eugenio Weinbaum,Actor


In [34]:
pd.merge(df1, df2)

,Nombre completo,Fecha Nacimiento,Profesion
0,Clotilde Acosta,3 octubre de 1940,Actriz
1,Rosario Bléfari,24/12/65,Cantante
2,Norberto Carredegoas,1936-04-12,Actor
3,Norberto Carredegoas,1936-04-12,Conductor
4,Norberto Carredegoas,1936-04-12,Publicista
5,Bárbara Torres,11-04-1973,Actuación
6,Eugenio Weinbaum,17/08/1961,Conductor
7,Eugenio Weinbaum,17/08/1961,Actor


**Join muchos a muchos**: Los joins de muchos a muchos pueden ser un poco confusos conceptualmente. Si la columna clave en ambos arreglos  contiene entradas duplicadas, entonces el resultado es un `merge` de muchos a muchos.

In [35]:
df1 = pd.DataFrame({'Empleado': ['Carlos', 'Esteban', 'María', 'Verónica'],
                    'Grupo': ['Contabilidad', 'Ingeniería', 'Ingeniería', 'People']})
df2 = pd.DataFrame({'Grupo': ['Contabilidad', 'Contabilidad', 'Ingeniería', 'Ingeniería', 'People', 'People'],
                    'Habilidades': ['Matemática', 'Planillas de cálculo', 'Programación', 'Linux', 'Planillas de cálculo', 'Organización']})

In [36]:
df1

,Empleado,Grupo
0,Carlos,Contabilidad
1,Esteban,Ingeniería
2,María,Ingeniería
3,Verónica,People


In [37]:
df2

,Grupo,Habilidades
0,Contabilidad,Matemática
1,Contabilidad,Planillas de cálculo
2,Ingeniería,Programación
3,Ingeniería,Linux
4,People,Planillas de cálculo
5,People,Organización


In [38]:
pd.merge(df1, df2)

,Empleado,Grupo,Habilidades
0,Carlos,Contabilidad,Matemática
1,Carlos,Contabilidad,Planillas de cálculo
2,Esteban,Ingeniería,Programación
3,Esteban,Ingeniería,Linux
4,María,Ingeniería,Programación
5,María,Ingeniería,Linux
6,Verónica,People,Planillas de cálculo
7,Verónica,People,Organización


### Especificando la llave

Ya hemos visto el comportamiento por defecto de `pd.merge()`, busca uno o más nombres de columnas coincidentes entre las dos entradas y utiliza esto como la clave. Sin embargo, muchas veces los nombres de las columnas no coinciden, y `pd.merge()` proporciona una variedad de opciones para manejar esto.

**El argumento `on`**: Se especificar explícitamente el nombre de la columna llave utilizando el argumento `on`, que toma un nombre de columna o una lista de nombres de columnas:

In [39]:
columns1 = ['Nombre completo', 'Fecha Nacimiento'] 
columns2 = ['Nombre completo', 'Ciudad', 'Profesion']

per_1a = ["Clotilde Acosta", "3 octubre de 1940"]
per_2a = ["Rosario Bléfari", "24/12/65"]
per_3a = ["Norberto Carredegoas", "1936-04-12"]
per_4a = ["Bárbara Torres", "11-04-1973"]
per_5a = ["Eugenio Weinbaum", "17/08/1961"]

per_1b = ["Norberto Carredegoas", "San Antonio Oeste"]
per_2b = ["Rosario Bléfari",  "MDP", "Cantante"]
per_3b = ["Clotilde Acosta", "Mar del Plata", "Actriz"]
per_4b = ["Bárbara Torres", "M del plata", "Actuación"]
per_5b = ["Eugenio Weinbaum", "MDQ", "Conductor"]

personajes1 = [per_1a, per_2a, per_3a, per_4a, per_5a]
personajes2 = [per_1b, per_2b, per_3b, per_4b, per_5b]

df1 = pd.DataFrame(personajes1, columns = columns1)
df2 = pd.DataFrame(personajes2, columns = columns2)

In [40]:
pd.merge(df1, df2, on="Nombre completo")

,Nombre completo,Fecha Nacimiento,Ciudad,Profesion
0,Clotilde Acosta,3 octubre de 1940,Mar del Plata,Actriz
1,Rosario Bléfari,24/12/65,MDP,Cantante
2,Norberto Carredegoas,1936-04-12,San Antonio Oeste,None
3,Bárbara Torres,11-04-1973,M del plata,Actuación
4,Eugenio Weinbaum,17/08/1961,MDQ,Conductor


**Los argumentos `left_on` y `right_on`**: En ocasiones, es posible que se desee combinar dos conjuntos de datos con nombres de columnas diferentes:

In [41]:
columns1 = ['Nombre completo', 'Fecha Nacimiento'] 
columns2 = ['Famoso', 'Ciudad', 'Profesion']

per_1a = ["Clotilde Acosta", "3 octubre de 1940"]
per_2a = ["Rosario Bléfari", "24/12/65"]
per_3a = ["Norberto Carredegoas", "1936-04-12"]
per_4a = ["Bárbara Torres", "11-04-1973"]
per_5a = ["Eugenio Weinbaum", "17/08/1961"]

per_1b = ["Norberto Carredegoas", "San Antonio Oeste"]
per_2b = ["Rosario Bléfari",  "MDP", "Cantante"]
per_3b = ["Clotilde Acosta", "Mar del Plata", "Actriz"]
per_4b = ["Bárbara Torres", "M del plata", "Actuación"]
per_5b = ["Eugenio Weinbaum", "MDQ", "Conductor"]

personajes1 = [per_1a, per_2a, per_3a, per_4a, per_5a]
personajes2 = [per_1b, per_2b, per_3b, per_4b, per_5b]

df1 = pd.DataFrame(personajes1, columns = columns1)
df2 = pd.DataFrame(personajes2, columns = columns2)

In [42]:
df1

,Nombre completo,Fecha Nacimiento
0,Clotilde Acosta,3 octubre de 1940
1,Rosario Bléfari,24/12/65
2,Norberto Carredegoas,1936-04-12
3,Bárbara Torres,11-04-1973
4,Eugenio Weinbaum,17/08/1961


In [43]:
df2

,Famoso,Ciudad,Profesion
0,Norberto Carredegoas,San Antonio Oeste,None
1,Rosario Bléfari,MDP,Cantante
2,Clotilde Acosta,Mar del Plata,Actriz
3,Bárbara Torres,M del plata,Actuación
4,Eugenio Weinbaum,MDQ,Conductor


In [44]:
pd.merge(df1, df2, left_on="Nombre completo", right_on="Famoso")

,Nombre completo,Fecha Nacimiento,Famoso,Ciudad,Profesion
0,Clotilde Acosta,3 octubre de 1940,Clotilde Acosta,Mar del Plata,Actriz
1,Rosario Bléfari,24/12/65,Rosario Bléfari,MDP,Cantante
2,Norberto Carredegoas,1936-04-12,Norberto Carredegoas,San Antonio Oeste,None
3,Bárbara Torres,11-04-1973,Bárbara Torres,M del plata,Actuación
4,Eugenio Weinbaum,17/08/1961,Eugenio Weinbaum,MDQ,Conductor


El resultado tiene una columna redundante que podemos eliminar si lo deseamos:

In [45]:
pd.merge(df1, df2, left_on="Nombre completo", right_on="Famoso").drop("Famoso", axis="columns")

,Nombre completo,Fecha Nacimiento,Ciudad,Profesion
0,Clotilde Acosta,3 octubre de 1940,Mar del Plata,Actriz
1,Rosario Bléfari,24/12/65,MDP,Cantante
2,Norberto Carredegoas,1936-04-12,San Antonio Oeste,None
3,Bárbara Torres,11-04-1973,M del plata,Actuación
4,Eugenio Weinbaum,17/08/1961,MDQ,Conductor


**Los argumentos `left_index` y `right_index`**: En casos muchos más raros, en vez de realizar el merge entre columnas, se puede usar los índices:

In [46]:
df1a = df1.set_index('Nombre completo')
df2a = df2.set_index('Famoso')

In [47]:
df1a

,Fecha Nacimiento
Nombre completo,
Clotilde Acosta,3 octubre de 1940
Rosario Bléfari,24/12/65
Norberto Carredegoas,1936-04-12
Bárbara Torres,11-04-1973
Eugenio Weinbaum,17/08/1961


In [48]:
df2a

,Ciudad,Profesion
Famoso,,
Norberto Carredegoas,San Antonio Oeste,None
Rosario Bléfari,MDP,Cantante
Clotilde Acosta,Mar del Plata,Actriz
Bárbara Torres,M del plata,Actuación
Eugenio Weinbaum,MDQ,Conductor


In [49]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,Fecha Nacimiento,Ciudad,Profesion
Clotilde Acosta,3 octubre de 1940,Mar del Plata,Actriz
Rosario Bléfari,24/12/65,MDP,Cantante
Norberto Carredegoas,1936-04-12,San Antonio Oeste,None
Bárbara Torres,11-04-1973,M del plata,Actuación
Eugenio Weinbaum,17/08/1961,MDQ,Conductor


Si se desea mezclar índices con columnas también podemos hacerlo:

In [50]:
pd.merge(df1a, df2, left_index=True, right_on="Famoso")

,Fecha Nacimiento,Famoso,Ciudad,Profesion
2,3 octubre de 1940,Clotilde Acosta,Mar del Plata,Actriz
1,24/12/65,Rosario Bléfari,MDP,Cantante
0,1936-04-12,Norberto Carredegoas,San Antonio Oeste,None
3,11-04-1973,Bárbara Torres,M del plata,Actuación
4,17/08/1961,Eugenio Weinbaum,MDQ,Conductor


## Especificando la aritmética del join

En todos los casos anteriores hemos pasado por alto una consideración importante al realizar un join: el tipo de aritmética de conjuntos utilizada en el join. Esto surge cuando un valor aparece en una columna llave pero no en la otra:

In [51]:
df3 = pd.DataFrame({'Nombre': ['Clotilde', 'Rosario', 'Norberto'],
                    'Ciudad': ['Mar del Plata', 'Mar del Plata', 'San Antonio Oeste']},
                   columns=['Nombre', 'Ciudad'])
df4 = pd.DataFrame({'Nombre': ['Clotilde', 'Eugenio'],
                    'Profesión': ['Actriz', 'Conductor']},
                   columns=['Nombre', 'Profesión'])

In [52]:
df3

,Nombre,Ciudad
0,Clotilde,Mar del Plata
1,Rosario,Mar del Plata
2,Norberto,San Antonio Oeste


In [53]:
df4

,Nombre,Profesión
0,Clotilde,Actriz
1,Eugenio,Conductor


In [54]:
pd.merge(df3, df4)

,Nombre,Ciudad,Profesión
0,Clotilde,Mar del Plata,Actriz


Aquí hemos mergeado dos conjuntos de datos que solo tienen una entrada `"Nombre"` en común: `Clotilde`. Por defecto, el resultado contiene la intersección de los dos conjuntos de entradas, esto es lo que se conoce como un `inner join`. Podemos especificar esto explícitamente usando el argumento `how`, que por defecto es `"inner"`:

In [55]:
pd.merge(df3, df4, how='inner')

,Nombre,Ciudad,Profesión
0,Clotilde,Mar del Plata,Actriz


Otras opciones para el argumento `how` son `'outer'`, `'left'` y `'right'`. Un `outer join` devuelve un join sobre la unión de las columnas de entrada y completa todos los valores faltantes con nulos:

In [56]:
pd.merge(df3, df4, how='outer')

,Nombre,Ciudad,Profesión
0,Clotilde,Mar del Plata,Actriz
1,Eugenio,NaN,Conductor
2,Norberto,San Antonio Oeste,NaN
3,Rosario,Mar del Plata,NaN


El `left join` y el `right join` devuelven joins sobre las entradas izquierdas y las entradas derechas, respectivamente:

In [57]:
pd.merge(df3, df4, how='left')

,Nombre,Ciudad,Profesión
0,Clotilde,Mar del Plata,Actriz
1,Rosario,Mar del Plata,NaN
2,Norberto,San Antonio Oeste,NaN


In [58]:
pd.merge(df3, df4, how='right')

,Nombre,Ciudad,Profesión
0,Clotilde,Mar del Plata,Actriz
1,Eugenio,NaN,Conductor


## Nombres de columnas superpuestos:

Finalmente, es posible nos encontremos en un caso donde los dos DataFrames de entrada tengan nombres de columnas conflictivos.

In [59]:
df5 = pd.DataFrame({'Nombre': ['Clotilde', 'Rosario', 'Norberto', 'Eugenio'],
                    'Importancia': [1, 2, 3, 4]})
df6 = pd.DataFrame({'Nombre': ['Clotilde', 'Rosario', 'Norberto', 'Eugenio'],
                    'Importancia': [3, 1, 4, 2]})

In [60]:
df5

,Nombre,Importancia
0,Clotilde,1
1,Rosario,2
2,Norberto,3
3,Eugenio,4


In [61]:
df6

,Nombre,Importancia
0,Clotilde,3
1,Rosario,1
2,Norberto,4
3,Eugenio,2


In [62]:
pd.merge(df5, df6, on="Nombre")

,Nombre,Importancia_x,Importancia_y
0,Clotilde,1,3
1,Rosario,2,1
2,Norberto,3,4
3,Eugenio,4,2


Debido a que el resultado tendría dos nombres de columna en conflicto, la función de merge automáticamente añade un sufijo `_x` o `_y` para hacer únicos los nombres de las columnas de salida. Es posible especificar un sufijo personalizado utilizando el argumento `suffixes`:

In [63]:
pd.merge(df5, df6, on="Nombre", suffixes=["_1990", "_1995"])

,Nombre,Importancia_1990,Importancia_1995
0,Clotilde,1,3
1,Rosario,2,1
2,Norberto,3,4
3,Eugenio,4,2
